In [60]:
import spacy, os
nlp = spacy.load('en_core_web_sm')
import pandas as pd
import numpy as np
import en_core_web_sm
nlp = en_core_web_sm.load()



    Only loading the 'en' tokenizer.



In [36]:
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(unicode(mod_word))

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return '1' # to handle the odd case of characters like 'x02', etc.

In [52]:
def get_closest(word, n):
    return [word]*n

In [54]:
cvs = pd.read_csv('prc_data.csv', sep='\t')
cvs = cvs.set_index('Unnamed: 0')

jd = pd.read_csv(os.getcwd()+'/data/jd_small.csv')
raw_description = jd['description'][5]
prc_description = ''

for word in raw_description.split():
    mod_word = modify(word)
    if (mod_word is not None):
        prc_description += (mod_word + ' ')

for word in prc_description.split():
    similar = get_closest(word, 2)
    print word, similar

successful [u'successful', u'successful']
software [u'software', u'software']
organisation [u'organisation', u'organisation']
base [u'base', u'base']
cheshire [u'cheshire', u'cheshire']
lookout [u'lookout', u'lookout']
skilled [u'skilled', u'skilled']
experience [u'experience', u'experience']
software [u'software', u'software']
developer [u'developer', u'developer']
join [u'join', u'join']
technical [u'technical', u'technical']
team [u'team', u'team']
involve [u'involve', u'involve']
software [u'software', u'software']
development [u'development', u'development']
lifecycle [u'lifecycle', u'lifecycle']
design [u'design', u'design']
implementation [u'implementation', u'implementation']
maintain [u'maintain', u'maintain']
current [u'current', u'current']
application [u'application', u'application']
website [u'website', u'website']
qualification [u'qualification', u'qualification']
minimum [u'minimum', u'minimum']
2 [u'2', u'2']
year [u'year', u'year']
commercial [u'commercial', u'commerci